In [1]:
!pip install pennylane scikit-learn numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 109.3 MB/s eta 0:00:00


In [13]:
# Kütüphanelerin yüklenmesi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine # WINE VERİ SETİ İÇE AKTARILDI
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.decomposition import PCA

import pennylane as qml
from pennylane import numpy as pnp
from sklearn.svm import SVC # Klasik SVM ve QSVM için çözücü

# --- YENİ VERİ YÜKLEME VE HAZIRLIK (WINE) ---
wine = load_wine()
X = wine.data
y = wine.target

# Wine verisi 13 özelliklidir. Simülatör kısıtlaması nedeniyle PCA ile indirgeme yapmalıyız.
n_features_original = X.shape[1]

# Qubit Kısıtlaması: 13 özelliği 4'e indirgeme
pca_q = PCA(n_components=4)
X_reduced = pca_q.fit_transform(X)

X_scaled = StandardScaler().fit_transform(X_reduced)
n_qubits = X_reduced.shape[1] # Qubit sayısı = 4

# Eğitim ve Test Setlerine Ayırma
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Orijinal Özellik Sayısı: {n_features_original}")
print(f"QSVM için Kullanılan Kübit Sayısı (PCA sonrası): {n_qubits}")
print(f"Eğitim Verisi Boyutu: {X_train.shape}, Test Verisi Boyutu: {X_test.shape}")

Orijinal Özellik Sayısı: 13
QSVM için Kullanılan Kübit Sayısı (PCA sonrası): 4
Eğitim Verisi Boyutu: (124, 4), Test Verisi Boyutu: (54, 4)


In [14]:
# --- Klasik Kernel SVM (RBF Çekirdeği) ---
print("--- Klasik Kernel SVM Başlatılıyor ---")

# RBF çekirdekli SVC kullanımı
svm_rbf = SVC(kernel='rbf', gamma='scale', random_state=42)
svm_rbf.fit(X_train, y_train)

y_pred_rbf = svm_rbf.predict(X_test)
acc_rbf = accuracy_score(y_test, y_pred_rbf)
f1_rbf = f1_score(y_test, y_pred_rbf, average='macro')

print(f"Klasik RBF Çekirdek SVM Doğruluk (Accuracy): {acc_rbf:.4f}")
print(f"Klasik RBF Çekirdek SVM F1 Skoru: {f1_rbf:.4f}")

# Klasik RBF Doğruluk skoru bu tabloda kullanılacaktır.
klasik_acc = acc_rbf

--- Klasik Kernel SVM Başlatılıyor ---
Klasik RBF Çekirdek SVM Doğruluk (Accuracy): 0.9444
Klasik RBF Çekirdek SVM F1 Skoru: 0.9467


In [15]:
# --- Kuantum Çekirdek Matrisi Hesaplama (QSVM İçin) ---
print("\n--- Kuantum Çekirdek Hesaplama ve QSVM Başlatılıyor ---")

dev = qml.device("default.qubit", wires=n_qubits)

def get_quantum_kernel_matrix(X_A, X_B):
    """X_A ve X_B matrisleri arasındaki kuantum çekirdek matrisini hesaplar."""
    n_samples_A = X_A.shape[0]
    n_samples_B = X_B.shape[0]
    kernel_matrix = np.zeros((n_samples_A, n_samples_B))

    for i in range(n_samples_A):
        for j in range(n_samples_B):

            # QNode tanımı (Angle Embedding ve Örtüşme Kullanımı)
            @qml.qnode(dev, interface="autograd")
            def overlap_qnode():
                # x_i'yi kodla
                qml.AngleEmbedding(X_A[i], wires=range(n_qubits))
                # x_j'yi ters kodla (örtüşme için)
                qml.adjoint(qml.AngleEmbedding)(X_B[j], wires=range(n_qubits))
                return qml.probs(wires=range(n_qubits))

            # |0...0> durumunun olasılığı (Kuantum Çekirdek Değeri)
            prob_0 = overlap_qnode()[0]
            kernel_matrix[i, j] = prob_0

    return kernel_matrix

# Çekirdek Matrislerinin Hesaplanması
K_train = get_quantum_kernel_matrix(X_train, X_train)
K_test = get_quantum_kernel_matrix(X_test, X_train)

# --- QSVM Uygulaması ---
svm_quantum = SVC(kernel='precomputed', random_state=42)
svm_quantum.fit(K_train, y_train)

# Tahmin yaparken K_test matrisini kullanıyoruz
y_pred_quantum = svm_quantum.predict(K_test)
acc_quantum = accuracy_score(y_test, y_pred_quantum)
f1_quantum = f1_score(y_test, y_pred_quantum, average='macro')

print(f"QSVM Doğruluk (Accuracy): {acc_quantum:.4f}")
print(f"QSVM F1 Skoru: {f1_quantum:.4f}")

# Kuantum Doğruluk skoru bu tabloda kullanılacaktır.
kuantum_acc = acc_quantum


--- Kuantum Çekirdek Hesaplama ve QSVM Başlatılıyor ---
QSVM Doğruluk (Accuracy): 0.9444
QSVM F1 Skoru: 0.9467
